In [22]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from warnings import filterwarnings
filterwarnings('ignore')

import torch._dynamo
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100
n = 6
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory
num_epochs = 100
lr = 0.01
print(device)

cuda


# Creating vocabulory

In [24]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:100]

In [25]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [26]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'<s> Question: {x} Answer: {y} </s>'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [27]:
vocab_list.add('<UNK>')

In [28]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

645

In [29]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([645, 10])

In [30]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([-1.5981, -0.4710,  1.0928,  0.2647, -0.8701,  0.3642,  1.0471,  0.9075,
        -0.2720,  1.2153])

# Creating the model

In [31]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=1024, vocab = vocab, dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        self.rev_vocab = {v:k for k,v in vocab.items()}
        vocab_len = len(vocab)
        self.n = n
        self.dim_embedding = dim_embedding
        self.embedding = nn.Embedding(vocab_len, dim_embedding).to(device)

        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)

    def forward(self,x):
        # x will be the indices of embedding representing the input words
        x = torch.tensor(x).to(device)
        # print(x.shape)
        x_embeddings = self.embedding(x).view(-1,(self.n-1)*self.dim_embedding).to(device)
        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        return out
    

    def generate(self, x):
        x = torch.tensor(x).to(device)

        # print(x.shape)
        # assert x.shape == 0 # fix shape here, and write a loop to do autoregressive text gen
        x = x[-(n-1):]
        # assert len(x) == self.n - 1
        logits = self.forward(x)
        probs = torch.softmax(logits, dim=1)
        prob, predicted_token_index = torch.max(probs, dim=1)
        return predicted_token_index

model = NeuralNetwork(n=n).to(device)

# Creating dataset

In [32]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(2958, 6)

In [33]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [34]:
model = model.to(device)
model = torch.compile(model)
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)


In [35]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        y_pred = model(x)
        loss = lossCategory(y_pred, y)
        if epoch%100==0 and i==0: 
            print(f'Epoch {epoch} Loss: {loss.item()}')
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


Epoch 0 Loss: 6.501012802124023


# Inference

In [36]:
text = '<s> Question: What is deep learning?'
tokenized_text = tokenize(text.lower())
embedding_indices = torch.tensor(np.array([vocab[word] for word in tokenized_text])).to(device)
embedding_indices

tensor([357, 631, 386, 143, 434, 623, 486, 464], device='cuda:0')

In [37]:
vocab_rev = {v:k for k,v in vocab.items()}

In [38]:
gen_word = ''
input_embeddings = embedding_indices
max_gen = 30
words_generated = 0
print(text, end = ' ')

while gen_word != '</s>' and words_generated <= max_gen:
    gen_word = model.generate(input_embeddings)
    words_generated += 1
    input_embeddings = list(input_embeddings)
    input_embeddings.append(gen_word.item())
    print(vocab_rev[gen_word.item()], end= ' ')



<s> Question: What is deep learning? answer : the learning is a type of machine learning that involves is the largest species of penguin ? answer : the african is the largest species of penguin ? answer 

In [39]:
# TO DO
# normalisation, standardise, proper init, weight decay, hyperparameter tunign, diff optimiser test, inference, earlyy stopping
# add direct connections, see if it is useful
# plot unigram, bi, tri, etc perplexity score, word error rate, etc